<a href="https://colab.research.google.com/github/Praxis-QR/BDSN/blob/main/KK_C4_Spark_RDD_Dataframe_SQL.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

![alt text](https://github.com/Praxis-QR/RDWH/raw/main/images/YantraJaalBanner.png)<br>


<hr>

[Prithwis Mukerjee](http://www.linkedin.com/in/prithwis)<br>

# Basic Data Analysis | Supermarket | Netflix | IMDB
An alternative approach is available at https://github.com/Praxis-QR/BDSN/blob/main/KK_C2A_Spark_WordCount_Alternative.ipynb

In [1]:
from datetime import datetime
import pytz
print('ॐ श्री सरस्वत्यै नमः',datetime.now(pytz.timezone('Asia/Calcutta')))

!python --version
!lsb_release -a

#check which version of MongoDB  is available
#!apt-cache policy mongodb

#check which versions of software are available
#!pip3 index versions pyspark
#!pip3 index versions pyngrok

ॐ श्री सरस्वत्यै नमः 2024-01-19 05:44:23.953587+05:30
Python 3.10.12
No LSB modules are available.
Distributor ID:	Ubuntu
Description:	Ubuntu 22.04.3 LTS
Release:	22.04
Codename:	jammy


# Download, Check Data
Sourced from https://github.com/search?q=owner%3Ajigsawlabs-student%20pyspark&type=repositories

In [2]:
import pandas as pd

#!wget -O netflix.csv "https://raw.githubusercontent.com/jigsawlabs-student/pyspark-cluster-lab/main/netflix_titles.csv"
!wget -q "https://github.com/Praxis-QR/BDSN/raw/main/Data2/NetFlix.csv"
#df = pd.read_csv('s3://jigsaw-labs-student/supermarket_sales.csv')
df1 = pd.read_csv('NetFlix.csv')

!wget -q 'https://github.com/Praxis-QR/BDSN/raw/main/Data2/SuperMarketSales.csv'
df2 = pd.read_csv('SuperMarketSales.csv')

#df2 = pd.read_csv("s3://jigsaw-labs-student/imdb_movies.csv")
!wget -q 'https://github.com/Praxis-QR/BDSN/raw/main/Data2/IMDBmovies.csv'
df3 = pd.read_csv('IMDBmovies.csv')
df1.shape, df2.shape, df3.shape

((7787, 12), (1000, 17), (2000, 7))

# Spark Installation


## Full Installation - Skip unless specifically required

In [ ]:
#!apt update > /dev/null
#!apt install openjdk-8-jdk-headless -qq > /dev/null

In [ ]:
# Get latest and correct version of Spark
#
# if the current version of Spark is not used, there may be errors
# check here for current versions http://apache.osuosl.org/spark
#
#!wget -q http://apache.osuosl.org/spark/spark-2.2.2/spark-2.2.2-bin-hadoop2.7.tgz
#!wget -q http://apache.osuosl.org/spark/spark-2.4.0/spark-2.4.0-bin-hadoop2.7.tgz
#!wget -q http://apache.osuosl.org/spark/spark-2.4.3/spark-2.4.3-bin-hadoop2.7.tgz
#!wget -q http://apache.osuosl.org/spark/spark-2.4.4/spark-2.4.4-bin-hadoop2.7.tgz
#!wget -q http://apache.osuosl.org/spark/spark-2.4.5/spark-2.4.5-bin-hadoop2.7.tgz
#!wget -q http://apache.osuosl.org/spark/spark-3.0.1/spark-3.0.1-bin-hadoop3.2.tgz
#!wget -q http://apache.osuosl.org/spark/spark-3.1.2/spark-3.1.2-bin-hadoop3.2.tgz

#!tar xf spark-2.4.5-bin-hadoop2.7.tgz
#!tar xf spark-3.0.1-bin-hadoop3.2.tgz
#!tar xf spark-3.1.2-bin-hadoop3.2.tgz
#!pip install -q findspark
#!pip install -q pyspark

#import os
#os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
#os.environ["SPARK_HOME"] = "/content/spark-2.4.0-bin-hadoop2.7"
#os.environ["SPARK_HOME"] = "/content/spark-2.4.4-bin-hadoop2.7"
#os.environ["SPARK_HOME"] = "/content/spark-2.4.5-bin-hadoop2.7"
#os.environ["SPARK_HOME"] = "/content/spark-3.0.1-bin-hadoop3.2"
#os.environ["SPARK_HOME"] = "/content/spark-3.1.2-bin-hadoop3.2"


In [ ]:
# findspark route is no more required
#import findspark
#findspark.init()
#
#findspark.init('/content/spark-2.2.2-bin-hadoop2.7')
#
# https://stackoverflow.com/questions/42223498/findspark-init-indexerror-list-index-out-of-range-error
#


## Simple pip install

In [3]:
!pip3 -q install pyspark
!pip3 -q install pyngrok
#!pip3 -q install s3fs

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 316.9/316.9 MB 3.2 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done


In [4]:
from pyspark.sql.functions import col, to_date, to_timestamp
from pyspark.sql.functions import regexp_replace
#from pyspark.sql.types import StructType,StructField, StringType, DateType, TimestampType, IntegerType,DoubleType, FloatType,NumericType, DecimalType
from pyspark.sql.types import *

In [5]:
#
from pyspark.sql import SparkSession
#
# spark = SparkSession.builder.master("local[*]").getOrCreate()
# note UI port switched from default 4040 to 4050 to avoid clash with ngrok
#
spark39 = SparkSession.builder.master("local[*]").config('spark.ui.port', '4050').getOrCreate()
#
sc = spark39.sparkContext
#
# The spark Console UI is available in the link that will be displayed in this cell.
# If you do not wish to use the Console, you may skip the Tunnel part
sc

<SparkContext master=local[*] appName=pyspark-shell>

##Tunnel with ngrok <br>
This part required ONLY if you need to access Spark Console<br>
To get you ngrok token please visit https://dashboard.ngrok.com/login <br>
and create a free account

###Loading ngrok token from GDrive<br>
this part may be skipped if ngrok can be placed directly in one of the two next steps

In [ ]:
from google.colab import drive
drive.mount('/content/drive')
#
#!ls /content/drive/'My Drive'/Praxis/WebCredentials/
#
!cp /content/drive/'My Drive'/Praxis/WebCredentials/ngrokToken.py .

Mounted at /content/drive


In [ ]:
# credential file for Prithwis Mukerjee
# this file needs to be uploaded into the VM

from ngrokToken import ngrokToken

#for the sake of privacy
#the following credentials need to be stored in a text file called ngrokToken.py
#in the format given below
#in the Colab VM

#otherwise, the values can be directly placed here

#ngrokToken = 'xxxxxxxxxxxxxxx'   # uncomment this line and place your own credentials here
#token is available at https://dashboard.ngrok.com/get-started/setup


#print(ngrokToken)
ngrokTokenCmd = 'ngrok authtoken '+ngrokToken

###ngrok - python wrapper <br>
this code adapted from https://towardsdatascience.com/quickly-share-ml-webapps-from-google-colab-using-ngrok-for-free-ae899ca2661a

In [ ]:
from pyngrok import ngrok
# you may place the token directly here
#!ngrok authtoken xxxxxxxxxxxxxxxxxxxxxxxxxxxxxx
get_ipython().system_raw(ngrokTokenCmd)
# Open a HTTP tunnel on the default port 80
#public_url = ngrok.connect(port = '4050')
public_url = ngrok.connect(4050)
#This is where the Spark Console UI will be visible
public_url

<NgrokTunnel: "https://0dd1-34-118-242-101.ngrok-free.app" -> "http://localhost:4050">

The next four cells are not required. Mainly used to check on the status of the Tunnel

In [ ]:
!curl -s http://localhost:4040/api/tunnels

{"tunnels":[{"name":"http-4050-7b6b18f4-59ba-42fd-8269-5f6dd6af7423","ID":"f95df2af638ea911dcbe2d0913061c4d","uri":"/api/tunnels/http-4050-7b6b18f4-59ba-42fd-8269-5f6dd6af7423","public_url":"https://0dd1-34-118-242-101.ngrok-free.app","proto":"https","config":{"addr":"http://localhost:4050","inspect":true},"metrics":{"conns":{"count":0,"gauge":0,"rate1":0,"rate5":0,"rate15":0,"p50":0,"p90":0,"p95":0,"p99":0},"http":{"count":0,"rate1":0,"rate5":0,"rate15":0,"p50":0,"p90":0,"p95":0,"p99":0}}}],"uri":"/api/tunnels"}


In [ ]:
!curl -s http://localhost:4040/api/tunnels | python3 -c \
    "import sys, json; print(json.load(sys.stdin)['tunnels'][0]['public_url'])"

https://0dd1-34-118-242-101.ngrok-free.app


In [ ]:
!ps

    PID TTY          TIME CMD
      1 ?        00:00:01 node
     17 ?        00:00:00 tail
     55 ?        00:00:01 jupyter-noteboo
     56 ?        00:00:00 dap_multiplexer
     66 ?        00:00:03 python3
     86 ?        00:00:00 python3
   1074 ?        00:00:09 java
   1171 ?        00:00:00 bash
   1172 ?        00:00:00 drive
   1173 ?        00:00:00 grep
   1220 ?        00:00:00 fusermount <defunct>
   1246 ?        00:00:00 drive
   1299 ?        00:00:00 bash
   1300 ?        00:00:00 tail
   1301 ?        00:00:00 python3
   1345 ?        00:00:00 ngrok
   1361 ?        00:00:00 ps


In [ ]:
!kill -9 1219

(7787, 12)

(1000, 17)

(2000, 7)

In [ ]:
#df.to_csv('SuperMarketSales.csv', sep=',', index=False, encoding='utf-8')
#df2.to_csv('IMDBmovies.csv', sep=',', index=False, encoding='utf-8')

# Supermarket Sales Data

In [6]:
!head SuperMarketSales.csv

Invoice ID,Branch,City,Customer type,Gender,Product line,Unit price,Quantity,Tax 5%,Total,Date,Time,Payment,cogs,gross margin percentage,gross income,Rating
750-67-8428,A,Yangon,Member,Female,Health and beauty,74.69,7,26.1415,548.9715,1/5/2019,13:08,Ewallet,522.83,4.761904762,26.1415,9.1
226-31-3081,C,Naypyitaw,Normal,Female,Electronic accessories,15.28,5,3.82,80.22,3/8/2019,10:29,Cash,76.4,4.761904762,3.82,9.6
631-41-3108,A,Yangon,Normal,Male,Home and lifestyle,46.33,7,16.2155,340.5255,3/3/2019,13:23,Credit card,324.31,4.761904762,16.2155,7.4
123-19-1176,A,Yangon,Member,Male,Health and beauty,58.22,8,23.288,489.048,1/27/2019,20:33,Ewallet,465.76,4.761904762,23.288,8.4
373-73-7910,A,Yangon,Normal,Male,Sports and travel,86.31,7,30.2085,634.3785,2/8/2019,10:37,Ewallet,604.17,4.761904762,30.2085,5.3
699-14-3026,C,Naypyitaw,Normal,Male,Electronic accessories,85.39,7,29.8865,627.6165,3/25/2019,18:30,Ewallet,597.73,4.761904762,29.8865,4.1
355-53-5943,A,Yangon,Member,Female,Electronic accesso

## RDD Operations

### Read CSV into RDD

In [7]:
sales_RDD1 = sc.textFile("SuperMarketSales.csv")
sales_RDD2 = spark39.read.text("SuperMarketSales.csv").rdd
# for an explanation of the alternate way to read text files
# see this https://stackoverflow.com/questions/52665353/reading-a-text-file-in-spark/52669632#52669632

In [8]:
sales_RDD1.take(5)

['Invoice ID,Branch,City,Customer type,Gender,Product line,Unit price,Quantity,Tax 5%,Total,Date,Time,Payment,cogs,gross margin percentage,gross income,Rating',
 '750-67-8428,A,Yangon,Member,Female,Health and beauty,74.69,7,26.1415,548.9715,1/5/2019,13:08,Ewallet,522.83,4.761904762,26.1415,9.1',
 '226-31-3081,C,Naypyitaw,Normal,Female,Electronic accessories,15.28,5,3.82,80.22,3/8/2019,10:29,Cash,76.4,4.761904762,3.82,9.6',
 '631-41-3108,A,Yangon,Normal,Male,Home and lifestyle,46.33,7,16.2155,340.5255,3/3/2019,13:23,Credit card,324.31,4.761904762,16.2155,7.4',
 '123-19-1176,A,Yangon,Member,Male,Health and beauty,58.22,8,23.288,489.048,1/27/2019,20:33,Ewallet,465.76,4.761904762,23.288,8.4']

In [9]:
sales_RDD2.take(5)

[Row(value='Invoice ID,Branch,City,Customer type,Gender,Product line,Unit price,Quantity,Tax 5%,Total,Date,Time,Payment,cogs,gross margin percentage,gross income,Rating'),
 Row(value='750-67-8428,A,Yangon,Member,Female,Health and beauty,74.69,7,26.1415,548.9715,1/5/2019,13:08,Ewallet,522.83,4.761904762,26.1415,9.1'),
 Row(value='226-31-3081,C,Naypyitaw,Normal,Female,Electronic accessories,15.28,5,3.82,80.22,3/8/2019,10:29,Cash,76.4,4.761904762,3.82,9.6'),
 Row(value='631-41-3108,A,Yangon,Normal,Male,Home and lifestyle,46.33,7,16.2155,340.5255,3/3/2019,13:23,Credit card,324.31,4.761904762,16.2155,7.4'),
 Row(value='123-19-1176,A,Yangon,Member,Male,Health and beauty,58.22,8,23.288,489.048,1/27/2019,20:33,Ewallet,465.76,4.761904762,23.288,8.4')]

In [10]:
!sed '1d' SuperMarketSales.csv > SuperMarketSales_NoHeader.csv
sales_RDD1 = sc.textFile("SuperMarketSales_NoHeader.csv")
sales_RDD2 = spark39.read.text("SuperMarketSales_NoHeader.csv").rdd
sales_RDD1.take(5)
#sales_RDD2.take(5)

['750-67-8428,A,Yangon,Member,Female,Health and beauty,74.69,7,26.1415,548.9715,1/5/2019,13:08,Ewallet,522.83,4.761904762,26.1415,9.1',
 '226-31-3081,C,Naypyitaw,Normal,Female,Electronic accessories,15.28,5,3.82,80.22,3/8/2019,10:29,Cash,76.4,4.761904762,3.82,9.6',
 '631-41-3108,A,Yangon,Normal,Male,Home and lifestyle,46.33,7,16.2155,340.5255,3/3/2019,13:23,Credit card,324.31,4.761904762,16.2155,7.4',
 '123-19-1176,A,Yangon,Member,Male,Health and beauty,58.22,8,23.288,489.048,1/27/2019,20:33,Ewallet,465.76,4.761904762,23.288,8.4',
 '373-73-7910,A,Yangon,Normal,Male,Sports and travel,86.31,7,30.2085,634.3785,2/8/2019,10:37,Ewallet,604.17,4.761904762,30.2085,5.3']

In [11]:
sales_RDD2.take(5)

[Row(value='750-67-8428,A,Yangon,Member,Female,Health and beauty,74.69,7,26.1415,548.9715,1/5/2019,13:08,Ewallet,522.83,4.761904762,26.1415,9.1'),
 Row(value='226-31-3081,C,Naypyitaw,Normal,Female,Electronic accessories,15.28,5,3.82,80.22,3/8/2019,10:29,Cash,76.4,4.761904762,3.82,9.6'),
 Row(value='631-41-3108,A,Yangon,Normal,Male,Home and lifestyle,46.33,7,16.2155,340.5255,3/3/2019,13:23,Credit card,324.31,4.761904762,16.2155,7.4'),
 Row(value='123-19-1176,A,Yangon,Member,Male,Health and beauty,58.22,8,23.288,489.048,1/27/2019,20:33,Ewallet,465.76,4.761904762,23.288,8.4'),
 Row(value='373-73-7910,A,Yangon,Normal,Male,Sports and travel,86.31,7,30.2085,634.3785,2/8/2019,10:37,Ewallet,604.17,4.761904762,30.2085,5.3')]

### Data Analysis with RDD

In [30]:
sales_RDD1.count()

1000

#### Filter Data

In [34]:
#normal_raw_data = raw_data.filter(lambda x: "normal." in x)
sales_wallet = sales_RDD1.filter(lambda x: "Ewallet" in x)
sales_cash = sales_RDD1.filter(lambda x: "Cash" in x)
sales_card = sales_RDD1.filter(lambda x: "Credit card" in x)
#
wallet_count = sales_wallet.count()
cash_count = sales_cash.count()
card_count = sales_card.count()
print(wallet_count, cash_count, card_count)

345 344 311


#### Aggregate

In [45]:
def x4m1(text):
    elements = text.split(",")
    return(int(elements[7]))


In [46]:
sales_wallet_Qty = sales_wallet.map(lambda x: x4m1(x))
sales_wallet_Qty.take(3)

[7, 8, 7]

In [48]:
wallet_total_Qty = sales_wallet_Qty.reduce(lambda x,y : x+y)
print('wallet total quantity = ',wallet_total_Qty, '\nwallet average quantity =', wallet_total_Qty/wallet_count)

wallet total quantity =  1892 
wallet average quantity = 5.484057971014493


#### Accumulators

In [55]:
wallet_data = sales_wallet_Qty.aggregate(
    (0,0), # the initial value
    (lambda acc, value: (acc[0] + value, acc[1] + 1)), # combine value with acc
    (lambda acc1, acc2: (acc1[0] + acc2[0], acc1[1] + acc2[1])) # combine accumulators
    )

print(wallet_data[0],wallet_data[1], '\nWallet Quantity Average = ', wallet_data[0]/wallet_data[1])

1892 345 
Wallet Quantity Average =  5.484057971014493


#### Two aggregations

In [56]:
def x4m2(text):
    elements = text.split(",")
    return(int(elements[7]), float(elements[15]))

In [57]:
sales_wallet_Qty_Inc = sales_wallet.map(lambda x: x4m2(x))
sales_wallet_Qty_Inc.take(3)

[(7, 26.1415), (8, 23.288), (7, 30.2085)]

In [60]:
wallet_data = sales_wallet_Qty_Inc.aggregate(
    (0,0,0), # the initial value
    (lambda acc, value: (acc[0] + value[0], acc[1] + value[1], acc[2] + 1)), # combine value with acc
    (lambda acc1, acc2, acc3 : (acc1[0] + acc2[0], acc2[1] + acc2[1], acc3[1] + acc3[1])) # combine accumulators
    )

print(wallet_data[0],wallet_data[2], '\nWallet Quantity Average = ', wallet_data[0]/wallet_data[2])

TypeError: <lambda>() missing 1 required positional argument: 'acc3'

In [53]:
wallet_total_Qty_Inc = sales_wallet_Qty_Inc.reduce(lambda x,y : x[0]+y[0])
print(wallet_total_Qty_Inc)

Py4JJavaError: An error occurred while calling z:org.apache.spark.api.python.PythonRDD.collectAndServe.
: org.apache.spark.SparkException: Job aborted due to stage failure: Task 1 in stage 49.0 failed 1 times, most recent failure: Lost task 1.0 in stage 49.0 (TID 61) (cedadc892f87 executor driver): org.apache.spark.api.python.PythonException: Traceback (most recent call last):
  File "/usr/local/lib/python3.10/dist-packages/pyspark/python/lib/pyspark.zip/pyspark/worker.py", line 1247, in main
    process()
  File "/usr/local/lib/python3.10/dist-packages/pyspark/python/lib/pyspark.zip/pyspark/worker.py", line 1239, in process
    serializer.dump_stream(out_iter, outfile)
  File "/usr/local/lib/python3.10/dist-packages/pyspark/python/lib/pyspark.zip/pyspark/serializers.py", line 274, in dump_stream
    vs = list(itertools.islice(iterator, batch))
  File "/usr/local/lib/python3.10/dist-packages/pyspark/rdd.py", line 1922, in func
    yield reduce(f, iterator, initial)
  File "/usr/local/l

### Convert RDD to Spark DataFrame

In [12]:
# Split data on , and convert string to other datatypes
#
sales_RDD1_split1 = sales_RDD1.map(lambda l: l.split(","))
#sales_RDD1_split1.take(1)
sales_RDD1_split2 = sales_RDD1_split1.map(lambda l: [l[0],l[1],l[2],l[3],l[4],l[5],float(l[6]),int(l[7]),float(l[8]),float(l[9]),l[10],l[11],l[12],float(l[13]),float(l[14]),float(l[15]),float(l[16])])
#sales_RDD1_split2.take(1)

In [13]:
# Convert to dataframe
# Date, Time does not convert correctly
#
colSchema = StructType([
    StructField('Invoice ID', StringType(), True),
    StructField('Branch', StringType(), True),
    StructField('City', StringType(), True),
    StructField('Customer Type', StringType(), True),
    StructField('Gender', StringType(), True),
    StructField('Product Line', StringType(), True),
    StructField('Unit Price', FloatType(), True),
    StructField('Quantity', IntegerType(), True),
    StructField('Tax 5%', DoubleType(), True),
    StructField('Total', DoubleType(), True),
    StructField('Date', StringType(), True),
    StructField('Time', StringType(), True),
    StructField('Payment', StringType(), True),
    StructField('cogs', DoubleType(), True),
    StructField('gross margin percentage', DoubleType(), True),
    StructField('gross income', DoubleType(), True),
    StructField('Rating', DoubleType(), True)
])

sales_df1 = spark39.createDataFrame(data = sales_RDD1_split2, schema = colSchema)

#sales_df1.printSchema()
#sales_df1.show(5)

In [14]:
# Fix Date, Time datatype
#
sales_df1 = sales_df1.withColumn('Date',to_date(col('Date'), "M/d/y"))
sales_df1 = sales_df1.withColumn('Time',to_timestamp(col('Time'), "HH:mm"))

sales_df1.printSchema()
sales_df1.show(5)

root
 |-- Invoice ID: string (nullable = true)
 |-- Branch: string (nullable = true)
 |-- City: string (nullable = true)
 |-- Customer Type: string (nullable = true)
 |-- Gender: string (nullable = true)
 |-- Product Line: string (nullable = true)
 |-- Unit Price: float (nullable = true)
 |-- Quantity: integer (nullable = true)
 |-- Tax 5%: double (nullable = true)
 |-- Total: double (nullable = true)
 |-- Date: date (nullable = true)
 |-- Time: timestamp (nullable = true)
 |-- Payment: string (nullable = true)
 |-- cogs: double (nullable = true)
 |-- gross margin percentage: double (nullable = true)
 |-- gross income: double (nullable = true)
 |-- Rating: double (nullable = true)

+-----------+------+---------+-------------+------+--------------------+----------+--------+-------+--------+----------+-------------------+-----------+------+-----------------------+------------+------+
| Invoice ID|Branch|     City|Customer Type|Gender|        Product Line|Unit Price|Quantity| Tax 5%|   To

## Dataframe Operations

### Read CSV into Spark Dataframe

In [15]:
sales_df2 = spark39.read.format("csv").option("header", "true").option("inferSchema", "true").load("./SuperMarketSales.csv")
#sales_df2.printSchema()
#sales_df2.show(5)

In [16]:
sales_df2 = sales_df2.withColumn('Date',to_date(col('Date'), "M/d/y"))
sales_df2.printSchema()
sales_df2.show(5)

root
 |-- Invoice ID: string (nullable = true)
 |-- Branch: string (nullable = true)
 |-- City: string (nullable = true)
 |-- Customer type: string (nullable = true)
 |-- Gender: string (nullable = true)
 |-- Product line: string (nullable = true)
 |-- Unit price: double (nullable = true)
 |-- Quantity: integer (nullable = true)
 |-- Tax 5%: double (nullable = true)
 |-- Total: double (nullable = true)
 |-- Date: date (nullable = true)
 |-- Time: timestamp (nullable = true)
 |-- Payment: string (nullable = true)
 |-- cogs: double (nullable = true)
 |-- gross margin percentage: double (nullable = true)
 |-- gross income: double (nullable = true)
 |-- Rating: double (nullable = true)

+-----------+------+---------+-------------+------+--------------------+----------+--------+-------+--------+----------+-------------------+-----------+------+-----------------------+------------+------+
| Invoice ID|Branch|     City|Customer type|Gender|        Product line|Unit price|Quantity| Tax 5%|   T

## SQL Table Operations

### Dataframe to Table

In [17]:
sales_df2.createOrReplaceTempView("tSales")
spark39.sql("describe tSales").show()

+--------------------+---------+-------+
|            col_name|data_type|comment|
+--------------------+---------+-------+
|          Invoice ID|   string|   NULL|
|              Branch|   string|   NULL|
|                City|   string|   NULL|
|       Customer type|   string|   NULL|
|              Gender|   string|   NULL|
|        Product line|   string|   NULL|
|          Unit price|   double|   NULL|
|            Quantity|      int|   NULL|
|              Tax 5%|   double|   NULL|
|               Total|   double|   NULL|
|                Date|     date|   NULL|
|                Time|timestamp|   NULL|
|             Payment|   string|   NULL|
|                cogs|   double|   NULL|
|gross margin perc...|   double|   NULL|
|        gross income|   double|   NULL|
|              Rating|   double|   NULL|
+--------------------+---------+-------+



In [18]:
spark39.sql("show tables").show()
spark39.sql("select * from tsales").show()

+---------+---------+-----------+
|namespace|tableName|isTemporary|
+---------+---------+-----------+
|         |   tsales|       true|
+---------+---------+-----------+

+-----------+------+---------+-------------+------+--------------------+----------+--------+-------+--------+----------+-------------------+-----------+------+-----------------------+------------+------+
| Invoice ID|Branch|     City|Customer type|Gender|        Product line|Unit price|Quantity| Tax 5%|   Total|      Date|               Time|    Payment|  cogs|gross margin percentage|gross income|Rating|
+-----------+------+---------+-------------+------+--------------------+----------+--------+-------+--------+----------+-------------------+-----------+------+-----------------------+------------+------+
|750-67-8428|     A|   Yangon|       Member|Female|   Health and beauty|     74.69|       7|26.1415|548.9715|2019-01-05|2024-01-19 13:08:00|    Ewallet|522.83|            4.761904762|     26.1415|   9.1|
|226-31-3081|

In [19]:
sql_drop_table = """drop table if exists praxisdb.sales_T"""

sql_drop_database = """drop database if exists praxisdb cascade"""

sql_create_database = """create database if not exists praxisdb location '/content/praxisdb/'"""

sql_create_table = """create table if not exists praxisdb.sales_T using parquet as select  * from tSales"""

spark39.sql(sql_drop_database)

spark39.sql(sql_create_database)

spark39.sql(sql_drop_table)

spark39.sql(sql_create_table)

DataFrame[]

In [20]:
spark39.sql("use praxisdb")
spark39.sql("show tables").show()

+---------+---------+-----------+
|namespace|tableName|isTemporary|
+---------+---------+-----------+
| praxisdb|  sales_t|      false|
|         |   tsales|       true|
+---------+---------+-----------+



In [21]:
spark39.sql("select * from praxisdb.sales_t").show()

+-----------+------+---------+-------------+------+--------------------+----------+--------+-------+--------+----------+-------------------+-----------+------+-----------------------+------------+------+
| Invoice ID|Branch|     City|Customer type|Gender|        Product line|Unit price|Quantity| Tax 5%|   Total|      Date|               Time|    Payment|  cogs|gross margin percentage|gross income|Rating|
+-----------+------+---------+-------------+------+--------------------+----------+--------+-------+--------+----------+-------------------+-----------+------+-----------------------+------------+------+
|750-67-8428|     A|   Yangon|       Member|Female|   Health and beauty|     74.69|       7|26.1415|548.9715|2019-01-05|2024-01-19 13:08:00|    Ewallet|522.83|            4.761904762|     26.1415|   9.1|
|226-31-3081|     C|Naypyitaw|       Normal|Female|Electronic access...|     15.28|       5|   3.82|   80.22|2019-03-08|2024-01-19 10:29:00|       Cash|  76.4|            4.761904762| 

### Table Data Analysis

In [29]:
# NOTE backtick ` used to enclose column names with spaces inside!
spark39.sql("select payment, count(*), sum(quantity), avg(quantity), sum(`gross income`), avg(`gross income`) from praxisdb.sales_t group by payment ").show()

+-----------+--------+-------------+-----------------+-----------------+------------------+
|    payment|count(1)|sum(quantity)|    avg(quantity)|sum(gross income)| avg(gross income)|
+-----------+--------+-------------+-----------------+-----------------+------------------+
|    Ewallet|     345|         1892|5.484057971014493|5237.767000000001|15.181933333333335|
|       Cash|     344|         1896|5.511627906976744|5343.170000000006|15.532470930232577|
|Credit card|     311|         1722|5.536977491961415|4798.432000000001| 15.42904180064309|
+-----------+--------+-------------+-----------------+-----------------+------------------+



In [28]:
spark39.sql("select `gross income` from praxisdb.sales_t").show()

+------------+
|gross income|
+------------+
|     26.1415|
|        3.82|
|     16.2155|
|      23.288|
|     30.2085|
|     29.8865|
|      20.652|
|       36.78|
|       3.626|
|       8.226|
|       2.896|
|       5.102|
|     11.7375|
|      21.595|
|       35.69|
|      28.116|
|     24.1255|
|      21.783|
|      8.2005|
|        4.03|
+------------+
only showing top 20 rows



#Chronobooks <br>
![alt text](https://1.bp.blogspot.com/-lTiYBkU2qbU/X1er__fvnkI/AAAAAAAAjtE/GhDR3OEGJr4NG43fZPodrQD5kbxtnKebgCLcBGAsYHQ/s600/Footer2020-600x200.png)<hr>
Chronotantra and Chronoyantra are two science fiction novels that explore the collapse of human civilisation on Earth and then its rebirth and reincarnation both on Earth as well as on the distant worlds of Mars, Titan and Enceladus. But is it the human civilisation that is being reborn? Or is it some other sentience that is revealing itself.
If you have an interest in AI and found this material useful, you may consider buying these novels, in paperback or kindle, from [http://bit.ly/chronobooks](http://bit.ly/chronobooks)